## Section IV.A Time Performance Evaluation

In [19]:
import src.gen_synthetic as gen_data
from scipy.fft import fft, fftfreq
from scipy import stats
import src.onlineSLE as utility
import time
import numpy as np
import periodicity_detection as period

In [20]:
sliding_window_size = 300
ts = gen_data.generate_syn1(residual_rate=0.05, is_export=False)['ts']
ts = stats.zscore(ts)


## Periodogram using scipy FFT

In [21]:
# num_iterations = len(ts) - window_size + 1
total_time = 0
results = []
W = ts[:sliding_window_size]
mathcal_F = fft(W)
xfreq, p_den = utility.peridogram(sliding_window_size, mathcal_F)
results.append(mathcal_F)
for idx, x_t in enumerate(ts[sliding_window_size:]):
    x_oldest = W[0]
    W = W[1:]
    W = np.append(W, x_t)
    ## start time
    start_time = time.time()
    mathcal_F = fft(W)
    xfreq, p_den = utility.peridogram(sliding_window_size, mathcal_F)
    end_time = time.time()
     ## end time
    total_time += (end_time - start_time)
    results.append(mathcal_F)

avg_time_ms = (total_time / idx) * 1000
print(f"Average execution time of Pden fft: {avg_time_ms} mil seconds")

Average execution time of Pden fft: 0.010798964896574875 mil seconds


## Periodogram using scipy SDFT

In [22]:
total_time = 0 
# num_iterations = len(ts) - window_size + 1
result = []
W = ts[:sliding_window_size]
mathcal_F = fft(W)
xfreq, p_den = utility.peridogram(sliding_window_size, mathcal_F)

result.append(mathcal_F)
N = len(mathcal_F)
k = np.arange(N)
twiddle = np.exp(2j * np.pi * k / N)
for idx, x_t in enumerate(ts[sliding_window_size:]):
    x_oldest = W[0]
    W = W[1:]
    W = np.append(W, x_t)
    ## start time
    start_time = time.time()
    start_fft = utility.update_sDFT(mathcal_F, x_oldest, x_t, twiddle)
    xfreq, p_den = utility.peridogram(sliding_window_size, mathcal_F)
    end_time = time.time()
    ## end time
    total_time += (end_time - start_time)
    result.append(start_fft)

avg_time_ms = (total_time / idx) * 1000
print(f"Average execution time of sdft (buffer): {avg_time_ms} mil seconds")

Average execution time of sdft (buffer): 0.0075868597612694395 mil seconds


## Autoperiod

In [23]:
total_time = 0  # To accumulate the total execution time of period.find_length(buffer)
#
for idx in range(len(ts) - sliding_window_size + 1):
    W = ts[idx:sliding_window_size + idx]
    start_time = time.time()
    SLE_result = period.autoperiod(W)
    end_time = time.time()
    total_time += (end_time - start_time)

avg_time_ms = (total_time / idx) * 1000
print(f"Average execution time of period.autoperiod(buffer): {avg_time_ms} msec")

Average execution time of period.autoperiod(buffer): 5.738927296229771 msec


## Find_length

In [24]:
total_time = 0  # To accumulate the total execution time of period.find_length(buffer)
#
for idx in range(len(ts) - sliding_window_size + 1):
    W = ts[idx:sliding_window_size + idx]
    start_time = time.time()
    SLE_result = period.find_length(W)
    end_time = time.time()
    total_time += (end_time - start_time)

avg_time_ms = (total_time / idx) * 1000
print(f"Average execution time of period.find_length(buffer): {avg_time_ms} msec")

Average execution time of period.autoperiod(buffer): 0.039895991889797915 msec


## OnlineSLE

In [25]:
total_time = 0
W = ts[:sliding_window_size]
mathcal_F = fft(W)
xfreq, periodogram_den = utility.peridogram(sliding_window_size, mathcal_F)
peak_index = utility.get_period_hints(periodogram_den)
for idx, x_t in enumerate(ts[sliding_window_size:]):
    x_oldest = W[0]
    W = W[1:]
    W = np.append(W, x_t)
    ## start time
    start_time = time.time()
    start_fft = utility.update_sDFT(mathcal_F, x_oldest, x_t, twiddle)
    xfreq, p_den = utility.peridogram(sliding_window_size, mathcal_F)
    peak_index = utility.get_period_hints(periodogram_den)
    end_time = time.time()
    ## end time
    total_time += (end_time - start_time)
    result.append(start_fft)

avg_time_ms = (total_time / idx) * 1000
print(f"Average execution time of OnlineSLE: {avg_time_ms} mil seconds")

Average execution time of OnlineSLE: 0.00930942061191239 mil seconds
